<a href="https://colab.research.google.com/github/shikhar00778/toxic-comment-classification/blob/master/Sentiment_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1558200265&Signature=sDXLIF7bM5jdxIr9CM%2BTrqc0gsTLSDCZDBDrL%2F6C0v3gsoowAjhu0x0s9%2FVSYwFk%2F%2BoXmCdpVlcSlgnWOgOYCu1dokohj2LQOuBtTeQMOcJCWG20X1uj6E1eDIGWhoxHJGY66DkHCFMEbTjgL9vK%2FLLOBFhw7MZ7wwTiCiZNxMTQ%2F45XFj7xx6sefyX%2BuLNfvAgiGH9dTzoZaDfs28fYMsPP4gG5xlwYNRER%2FgNSIQR7X3xmIbPb2W53dvSiRzFTG4hCwXaeZwXETTNW8l%2Fv5WGAN8cVSKha8K9S6n6e0I5Nny5SNpQuHM7pOctUxE2zqpuV9SXMhVr99dFKIolEkA%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-toxic-comment-classification-challenge.zip" -O "jigsaw-toxic-comment-classification-challenge.zip" -c

--2019-05-17 15:48:44--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1558200265&Signature=sDXLIF7bM5jdxIr9CM%2BTrqc0gsTLSDCZDBDrL%2F6C0v3gsoowAjhu0x0s9%2FVSYwFk%2F%2BoXmCdpVlcSlgnWOgOYCu1dokohj2LQOuBtTeQMOcJCWG20X1uj6E1eDIGWhoxHJGY66DkHCFMEbTjgL9vK%2FLLOBFhw7MZ7wwTiCiZNxMTQ%2F45XFj7xx6sefyX%2BuLNfvAgiGH9dTzoZaDfs28fYMsPP4gG5xlwYNRER%2FgNSIQR7X3xmIbPb2W53dvSiRzFTG4hCwXaeZwXETTNW8l%2Fv5WGAN8cVSKha8K9S6n6e0I5Nny5SNpQuHM7pOctUxE2zqpuV9SXMhVr99dFKIolEkA%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-toxic-comment-classification-challenge.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55956017 (53M) [application/zip]
Saving to: ‘jigsaw-toxic-comment-classification-cha

In [2]:
!ls

jigsaw-toxic-comment-classification-challenge.zip  sample_data


In [3]:
!unzip jigsaw-toxic-comment-classification-challenge.zip

Archive:  jigsaw-toxic-comment-classification-challenge.zip
  inflating: test_labels.csv         
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('train.csv',encoding="ISO-8859-1")

In [144]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
df = df.drop(['id'] , axis = 1)

In [146]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
import torch.utils.data
import sklearn.metrics
import torch
import pandas as pd
from torch.utils.data import Dataset
import tqdm
import spacy
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import fastai
from keras.preprocessing import text,sequence

In [0]:
from fastai.train import Learner
from fastai.train import DataBunch

In [0]:
df['comment_text'] = df['comment_text'].fillna('')
df['toxic'] = df['toxic'].fillna('0')
df['severe_toxic'] = df['severe_toxic'].fillna('0')
df['obscene'] = df['obscene'].fillna('0')
df['threat'] = df['threat'].fillna('0')
df['insult'] = df['insult'].fillna('0')
df['identity_hate'] = df['identity_hate'].fillna('0')

In [148]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [60]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from spacy.lang.en.stop_words import STOP_WORDS

In [0]:
exceptionStopWords = {
    'again',
    'against',
    'ain',
    'almost',
    'among',
    'amongst',
    'amount',
    'anyhow',
    'anyway',
    'aren',
    "aren't",
    'below',
    'bottom',
    'but',
    'cannot',
    'couldn',
    "couldn't",
    'didn',
    "didn't",
    'doesn',
    "doesn't",
    'don',
    "don't",
    'done',
    'down',
    'except',
    'few',
    'hadn',
    "hadn't",
    'hasn',
    "hasn't",
    'haven',
    "haven't",
    'however',
    'isn',
    "isn't",
    'least',
    'mightn',
    "mightn't",
    'move',
    'much',
    'must',
    'mustn',
    "mustn't",
    'needn',
    "needn't",
    'neither',
    'never',
    'nevertheless',
    'no',
    'nobody',
    'none',
    'noone',
    'nor',
    'not',
    'nothing',
    'should',
    "should've",
    'shouldn',
    "shouldn't",
    'too',
    'top',
    'up',
    'very'
    'wasn',
    "wasn't",
    'well',
    'weren',
    "weren't",
    'won',
    "won't",
    'wouldn',
    "wouldn't",
}

In [0]:
stop_words = set(stop_words).union(STOP_WORDS)

In [0]:
final_stop_words = stop_words-exceptionStopWords

In [0]:
CONTRACTION_MAP = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 

In [0]:
def expand_contractions(sentence, contraction_mapping): 
     
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),  
                                      flags=re.IGNORECASE|re.DOTALL) 
    def expand_match(contraction): 
        match = contraction.group(0) 
        first_char = match[0] 
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())                        
        expanded_contraction = first_char+expanded_contraction[1:] 
        return expanded_contraction 
         
    expanded_sentence = contractions_pattern.sub(expand_match, sentence) 
    return expanded_sentence

In [0]:
from nltk.tokenize import sent_tokenize

In [67]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
import gensim
import re
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem.lancaster import LancasterStemmer
lc = LancasterStemmer()
from nltk.stem import SnowballStemmer
sb = SnowballStemmer("english")

In [69]:
l = []

for row,index in tqdm.tqdm(df.iterrows()):
    #print(l[i])
    s = [expand_contractions(txt, CONTRACTION_MAP)  
                     for txt in sent_tokenize(index[0])]
    
    #print(s)
    #print(l[i])
    
    res = []
    sample = ' '.join(s)
    sample = sample.split()
    #print(sample)
    for j in sample:
      if j not in final_stop_words:
        res.append(j)
    #break
    l.append(' '.join(res))    
    #print(l)
    #break

159571it [02:53, 922.21it/s]


In [0]:
res = []
for text in l:
  punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '"“”’=' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'+'\n'
  for p in punct:
    text = text.replace(p, ' ')
  res.append(text)

In [0]:
preprocessed_data = pd.DataFrame(res)

In [149]:
preprocessed_data.head()

,0
0,Explanation Why edits username Hardcore Metall...
1,D aww He matches background colour I seemingl...
2,Hey man I not trying edit war It guy constan...
3,More I cannot real suggestions improvement ...
4,You sir hero Any chance remember page on


In [0]:
df['comment_text'] = preprocessed_data[0]

In [151]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation Why edits username Hardcore Metall...,0,0,0,0,0,0
1,D aww He matches background colour I seemingl...,0,0,0,0,0,0
2,Hey man I not trying edit war It guy constan...,0,0,0,0,0,0
3,More I cannot real suggestions improvement ...,0,0,0,0,0,0
4,You sir hero Any chance remember page on,0,0,0,0,0,0


In [153]:
arr = df.columns

classes = arr[1:]
classes = np.asarray(classes)
classes = classes.tolist()

classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [0]:
def make_data(df):
  data=[]
  lab=[]
  for row,index in df.iterrows():
    temp=[]
    for class_name in classes:
      if index[class_name]==1:
        temp.append(class_name)
    if len(temp)==0: 
      temp.append("no_class")
    temp = ' '.join(temp)
    lab.append(temp)
    data.append(index["comment_text"])
  X = pd.DataFrame({"labels":lab,"comment_text":data})
  return X

In [155]:

df_clean = make_data(df)
df_clean.head()

,labels,comment_text
0,no_class,Explanation Why edits username Hardcore Metall...
1,no_class,D aww He matches background colour I seemingl...
2,no_class,Hey man I not trying edit war It guy constan...
3,no_class,More I cannot real suggestions improvement ...
4,no_class,You sir hero Any chance remember page on


In [157]:
df_clean.isna().any()


labels          False
comment_text    False
dtype: bool

In [0]:
df_clean.to_csv('cleane_train.csv',index=False)

In [0]:
df_temp = pd.read_csv("cleane_train.csv")

In [161]:
df_temp.isna().any()


labels          False
comment_text     True
dtype: bool

In [0]:
from fastai.text import *
path = '.'
data_lm = TextDataBunch.from_df(path, train_df = df_clean[:140000], valid_df = df_clean[140000:], text_cols=1,max_vocab=60000,min_freq=2, label_cols=0,label_delim=" ")

In [165]:
data_lm.show_batch()

text,target
xxbos xxmaj take that xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass,obscene;severe_toxic;toxic
xxbos unblock xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron xxmaj oxymoron xxmaj oxymoron moron,toxic
xxbos xxmaj amuca xxunk xxmaj the xxmaj amuca tribe xxmaj turkish xxmaj amuca xxmaj kabilesi xxmaj amucalar nomadic xxmaj oghuz xxmaj turks tribes xxmaj ottoman xxmaj empire moved xxmaj anatolia xxmaj balkans xxmaj at end 14th xxmaj century clan established settled life xxmaj ottoman xxmaj thrace xxmaj today live xxunk xxmaj province xxmaj the founded village tribe xxunk located xxmaj xxunk xxunk xxmaj meaning word xxmaj amuca xxmaj uncle old,no_class
xxbos xxmaj the xxmaj starting xxmaj point near xxmaj thebes xxmaj in xxmaj exodus xxmaj ramesses pair treasure cities emporia station xxmaj exodus xxmaj in story treasure cities xxmaj pithom xxmaj delta xxmaj ra mes ses xxmaj thebes xxmaj egypt xxmaj exodus 1 7 14 xxmaj and children xxmaj israel fruitful increased abundantly multiplied xxunk exceeding mighty land filled them xxmaj now arose up new king xxmaj egypt knew not,no_class
xxbos xxup ok xxup let xxup me xxup say xxup one xxup thing xxup fuck xxup you xxup all xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup you xxup fuck xxup,insult;obscene;severe_toxic;toxic


In [0]:
learn = text_classifier_learner(data_lm, drop_mult=0.5, arch=AWD_LSTM, metrics=accuracy_thresh)
#learn.load_encoder('fine_tuned_enc')
learn.freeze()

In [44]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [50]:
data_lm.show_batch()

text,target
xxbos no_class obscene threat insult identity_hate,176
xxbos no_class obscene threat insult identity_hate,143260
xxbos severe_toxic obscene insult identity_hate,27198
xxbos no_class obscene insult identity_hate,39574
xxbos no_class obscene insult identity_hate,72593


In [45]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy_thresh,time


KeyboardInterrupt: ignored